### <u>Homework 2: Map/Reduce</u>
- By Thomas Truong

<b>The Data</b>
- There is a data file that needs to be analyzed by a Map/Reduce system.  The file's name is 'cs4650hw2.dat', and it is available on Canvas (Look in Modules > Extra Files > Homework 2.
- The data represents a 2 dimensional array of cells, and there are several values in each cell. The basic task is this: for each row in the array, examine all of the values in all of the cells of that row, returning the smallest value found. At the same time, for each column in the array, examine all of the values in all of the cells in that column, returning the largest value found.
- Now for the details:
  - The columns are labeled 'A' through 'J', so there are 10 columns.
  - The rows are labeled 'K' through 'T', so there are 10 rows.
  - The values are integers from 0 to 999.
  - The data file uses CSV format, with each line of the file giving the column name, the row name, and one of the values for that cell.  For example, some of the lines of the file might be:
    - A,K,652
    - A,K,378
    - A,L,21
    - E,M,411

__Part 1__
- Write a map reduce program that will examine this file, then print, for each column, the name of the column and the largest value found in that column, and for each row, the name of the row and the smallest value found in that row.  Note that the column and row names are strings, a single character.  The output should be of the form:
  - "A", 564
  - "B", 329
- There should be 20 lines in the report, and they do not have to be in any particular order.

In [25]:
%%file row_column_low_high.py
# Step 0: create a new file (lower cased + underscored).
"""row_column_low_high.py

For every column, print high; for every row, print low.
"""

# Step 1: import MRJob.
from mrjob.job import MRJob


# Step 2: create a class that inherits from MRJob.
class RowColumnLowHigh(MRJob):
  """Extracts data from a file.
  
  Prints the row/column with the low/high respectively.
  """

  # Step 3: create mapper (input: file -> data).
  def mapper(self, _, line):
    """Extracts data from a line and maps it."""

    # Clean up the data.
    cleanedLine = line.replace(" ", "")      # Remove whitespaces.
    cleanedLine = cleanedLine.split(",")     # Split into array.
    cleanedLine[2] = int(cleanedLine[2])     # Make int.

    # Check for valid data.
    if (not cleanedLine[0].isalpha()):
      return
    if (not cleanedLine[1].isalpha()):
      return
    if (cleanedLine[2] < 0 or cleanedLine[2] > 999):
      return

    # Cleaning part 2: make uppercase if needed.
    cleanedLine[0] = cleanedLine[0].upper()
    cleanedLine[1] = cleanedLine[1].upper()

    # Check part 2: A-T only.
    if (ord(cleanedLine[1]) > 84):
      return

    # Return column/row with number.
    yield cleanedLine[0], cleanedLine[2]
    yield cleanedLine[1], cleanedLine[2]


  # Step 4: create reducer (output: data -> console/file/etc).
  def reducer(self, key, values):
    """Prints the row/column with the low/high respectively.
    
    For every column, print high.
    For every row, print low.
    """

    # Convert and check letter based on ASCII.
    asciiKey = ord(key)
    # A-J = column (max).
    if (asciiKey >= 65 and asciiKey <= 74):
      yield key, max(values)
    # K-T = row (min).
    else:
      yield key, min(values)


# Step 5: set up main to run program.
if __name__ == "__main__":
  RowColumnLowHigh.run()


Overwriting row_column_low_high.py


In [26]:
!python row_column_low_high.py --no-bootstrap-mrjob cs4650hw1.dat

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/row_column_low_high.thomas.20231007.044518.131500
Running step 1 of 1...
job output is in /tmp/row_column_low_high.thomas.20231007.044518.131500/output
Streaming final output from /tmp/row_column_low_high.thomas.20231007.044518.131500/output...
"R"	2
"J"	992
"T"	2
"F"	997
"A"	994
"P"	12
"G"	997
"S"	5
"I"	995
"N"	13
"E"	998
"C"	987
"D"	995
"H"	997
"M"	0
"L"	3
"K"	0
"O"	1
"Q"	0
"B"	999
Removing temp directory /tmp/row_column_low_high.thomas.20231007.044518.131500...


__Part 2__
- This is a second map reduce program that is similar to the first part, but has an additional feature.  For a column, the output should show the column name, the largest value in that column, and the name of one of the rows where this maximum value is found (there may be more than one cell with the same maximum value).  For each row, the output should show the row name, the smallest value in that row, and the name of one of the columns where this minimum value is found.
- Again, the output should have 20 lines, and they do not have to be in any order.  An example of one line of the report is:
  - "A", {"value":564, "example":"M"}

In [27]:
%%file row_column_min_max.py
# Step 0: create a new file (lower cased + underscored).
"""row_column_min_max.py

For every column, print max and the row(s) that have the max.
For every row, print the min and the column(s) that have the min.
"""

# Step 1: import MRJob.
from mrjob.job import MRJob


# Step 2: create a class that inherits from MRJob.
class RowColumnMinMax(MRJob):
  """Extracts data from a file.

  Prints the column with the largest value,
    and the row(s) that have the maximum value.
  Prints the row with the smallest value,
    and the column(s) that have the minimum value.
  """

  # Step 3: create mapper (input: file -> data).
  def mapper(self, _, line):
    """Extracts data from a line and maps it."""

    # Clean up the data.
    cleanedLine = line.replace(" ", "")      # Remove whitespaces.
    cleanedLine = cleanedLine.split(",")     # Split into array.
    cleanedLine[2] = int(cleanedLine[2])     # Make int.

    # Check for valid data.
    if (not cleanedLine[0].isalpha()):
      return
    if (not cleanedLine[1].isalpha()):
      return
    if (cleanedLine[2] < 0 or cleanedLine[2] > 999):
      return

    # Cleaning part 2: make uppercase if needed.
    cleanedLine[0] = cleanedLine[0].upper()
    cleanedLine[1] = cleanedLine[1].upper()

    # Check part 2: A-T only.
    if (ord(cleanedLine[1]) > 84):
      return

    # Return (column, row) with number.
    yield cleanedLine[0], [cleanedLine[1], cleanedLine[2]]
    yield cleanedLine[1], [cleanedLine[0], cleanedLine[2]]


  # Step 4: create reducer (output: data -> console/file/etc).
  def reducer(self, key, values):
    """Outputs a formatted result.
    
    Prints the column with the largest value,
      and the row(s) that has the maximum value.
    Prints the row with the smallest value,
      and the column(s) that have the minimum value.
    """

    # Converts key to ASCII.
    asciiKey = ord(key)
    # Indicates whether key is a column or a row.
    isKeyColumn = (asciiKey >= 65 and asciiKey <= 74)

    min_max = next(values)

    # Print based on column/row.
    if (isKeyColumn):
      # Is column, get max.
      for v in values:
        if v[1] > min_max[1]:
          min_max = v
      yield key, {"value": min_max[1], "row": min_max[0]}
    else:
      # Is row, get min.
      for v in values:
        if v[1] < min_max[1]:
          min_max = v
      yield key, {"value": min_max[1], "col": min_max[0]}


# Step 5: set up main to run program.
if __name__ == "__main__":
  RowColumnMinMax.run()


Overwriting row_column_min_max.py


In [28]:
!python row_column_min_max.py --no-bootstrap-mrjob cs4650hw1.dat

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/row_column_min_max.thomas.20231007.044524.855220
Running step 1 of 1...
job output is in /tmp/row_column_min_max.thomas.20231007.044524.855220/output
Streaming final output from /tmp/row_column_min_max.thomas.20231007.044524.855220/output...
"R"	{"value": 2, "col": "F"}
"J"	{"value": 992, "row": "K"}
"T"	{"value": 2, "col": "G"}
"F"	{"value": 997, "row": "M"}
"A"	{"value": 994, "row": "R"}
"P"	{"value": 12, "col": "B"}
"G"	{"value": 997, "row": "M"}
"S"	{"value": 5, "col": "I"}
"I"	{"value": 995, "row": "S"}
"N"	{"value": 13, "col": "B"}
"E"	{"value": 998, "row": "N"}
"C"	{"value": 987, "row": "S"}
"D"	{"value": 995, "row": "M"}
"H"	{"value": 997, "row": "P"}
"M"	{"value": 0, "col": "E"}
"L"	{"value": 3, "col": "H"}
"K"	{"value": 0, "col": "H"}
"O"	{"value": 1, "col": "H"}
"Q"	{"value": 0, "col": "C"}
"B"	{"value": 999, "row": "N"}
Removing temp directory /tmp/row_co

__Part 3__
- This is a third map reduce program that is a further extension of Part 2.  In this case, for the output, we want to include a list of all rows/columns that have that maximum/minimum value.  Note that most rows and columns will have a unique extreme, but some will have duplicates.  This output might look like this:
  - "A", ["value":564, "examples": ["M"]}
  - "D", ["value":437, "examples": ["L",  "P"]}

In [29]:
%%file rows_columns_min_max.py
# Step 0: create a new file (lower cased + underscored).
"""rows_columns_min_max.py

For every column, print max and the row(s) that have the max.
For every row, print the min and the column(s) that have the min.
"""

# Step 1: import MRJob.
from mrjob.job import MRJob


# Step 2: create a class that inherits from MRJob.
class RowsColumnsMinMax(MRJob):
  """Extracts data from a file.

  Prints the column with the largest value,
    and the row(s) that have the maximum value.
  Prints the row with the smallest value,
    and the column(s) that have the minimum value.
  """

  # Step 3: create mapper (input: file -> data).
  def mapper(self, _, line):
    """Extracts data from a line and maps it."""

    # Clean up the data.
    cleanedLine = line.replace(" ", "")      # Remove whitespaces.
    cleanedLine = cleanedLine.split(",")     # Split into array.
    cleanedLine[2] = int(cleanedLine[2])     # Make int.

    # Check for valid data.
    if (not cleanedLine[0].isalpha()):
      return
    if (not cleanedLine[1].isalpha()):
      return
    if (cleanedLine[2] < 0 or cleanedLine[2] > 999):
      return

    # Cleaning part 2: make uppercase if needed.
    cleanedLine[0] = cleanedLine[0].upper()
    cleanedLine[1] = cleanedLine[1].upper()

    # Check part 2: A-T only.
    if (ord(cleanedLine[1]) > 84):
      return

    # Return (column, row) with number.
    yield cleanedLine[0], [cleanedLine[1], cleanedLine[2]]
    yield cleanedLine[1], [cleanedLine[0], cleanedLine[2]]


  # Step 4: create reducer (output: data -> console/file/etc).
  def reducer(self, key, values):
    """Outputs a formatted result.
    
    Prints the column with the largest value,
      and the row(s) that has the maximum value.
    Prints the row with the smallest value,
      and the column(s) that have the minimum value.
    """

    # Converts key to ASCII.
    asciiKey = ord(key)
    # Indicates whether key is a column or a row.
    isKeyColumn = (asciiKey >= 65 and asciiKey <= 74)

    # Tracker variables.
    current_iterator = next(values)
    current_value = current_iterator[1]
    value_list = [current_iterator[0]]

    # For every value in values.
    for v in values:
      # Equal, append to value_list.
      if v[1] == current_value:
        value_list.append(v[0])
      # Is a column, check for max.
      elif (isKeyColumn):
        # v[1] is new max.
        if v[1] > current_value:
          current_value = v[1]
          value_list = [v[0]]  # Reset list to the new max only.
      # Is a row, check for min.
      else:
        # v[1] is new min.
        if v[1] < current_value:
          current_value = v[1]
          value_list = [v[0]]  # Reset list to the new min only.
    
    # Print based on column/row.
    if (isKeyColumn):
      yield key, {"value": current_value, "row(s)": value_list}
    else:
      yield key, {"value": current_value, "col(s)": value_list}


# Step 5: set up main to run program.
if __name__ == "__main__":
  RowsColumnsMinMax.run()


Overwriting rows_columns_min_max.py


In [30]:
!python rows_columns_min_max.py --no-bootstrap-mrjob cs4650hw1.dat

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/rows_columns_min_max.thomas.20231007.044530.890170
Running step 1 of 1...
job output is in /tmp/rows_columns_min_max.thomas.20231007.044530.890170/output
Streaming final output from /tmp/rows_columns_min_max.thomas.20231007.044530.890170/output...
"R"	{"value": 2, "col(s)": ["F"]}
"J"	{"value": 992, "row(s)": ["K"]}
"T"	{"value": 2, "col(s)": ["G"]}
"F"	{"value": 997, "row(s)": ["M"]}
"A"	{"value": 994, "row(s)": ["R", "P"]}
"P"	{"value": 12, "col(s)": ["B"]}
"G"	{"value": 997, "row(s)": ["M"]}
"S"	{"value": 5, "col(s)": ["I"]}
"I"	{"value": 995, "row(s)": ["S"]}
"N"	{"value": 13, "col(s)": ["B"]}
"E"	{"value": 998, "row(s)": ["N", "S"]}
"C"	{"value": 987, "row(s)": ["S"]}
"D"	{"value": 995, "row(s)": ["M"]}
"H"	{"value": 997, "row(s)": ["P"]}
"M"	{"value": 0, "col(s)": ["E"]}
"L"	{"value": 3, "col(s)": ["H", "B"]}
"K"	{"value": 0, "col(s)": ["H"]}
"O"	{"value": 1, "